In [1]:
import numpy as np
from aocd import get_data, submit
from operator import itemgetter
import re
import collections

In [2]:
example_data = "....#..\n..###.#\n#...#.#\n.#...##\n#.###..\n##.#.##\n.#..#.."
example_data2 = ".....\n..##.\n..#..\n.....\n..##.\n....."
real_data = get_data(day=23, year=2022)

In [3]:
def add_padding(grid):
    pad_fr = 1 if any(grid[0] == "#") else 0
    pad_lr = 1 if any(grid[-1] == "#") else 0
    pad_fc = 1 if any(grid[:,0] == "#") else 0
    pad_lc = 1 if any(grid[:,-1] == "#") else 0

    grid = np.pad(grid, pad_width=((pad_fr, pad_lr),(pad_fc, pad_lc)), constant_values=".")

    return grid

def has_to_move(position, grid):
    return len(np.argwhere(grid[position[0]-1:position[0]+2,position[1]-1:position[1]+2] == "#")) > 1

def get_next_position(round, position, grid):

    options = [
        (not any(grid[position[0]-1,position[1]-1:position[1]+2] == "#"), (position[0]-1, position[1])),
        (not any(grid[position[0]+1,position[1]-1:position[1]+2] == "#"), (position[0]+1, position[1])),
        (not any(grid[position[0]-1:position[0]+2,position[1]-1] == "#"), (position[0], position[1]-1)),
        (not any(grid[position[0]-1:position[0]+2,position[1]+1] == "#"), (position[0], position[1]+1))
    ]

    for i in range(4):
        if options[(i + round) % 4][0]:
            return options[(i + round) % 4][1]

    return None



In [4]:
grid = np.array([list(line) for line in real_data.splitlines()])

for round in range(1000):
    grid = add_padding(grid)
    positions = list(map(tuple, np.argwhere(grid == "#")))

    no_position_proposed = True
    proposed_positions = []
    for position in map(tuple, positions):
        if has_to_move(position, grid):
            new_position = get_next_position(round, position, grid)

            if new_position is not None:
                proposed_positions.append(new_position)
                no_position_proposed = False
            else:
                proposed_positions.append(position)
        else:
            proposed_positions.append(position)

    if no_position_proposed:
        break

    grid = np.full_like(grid, fill_value=".")

    for index, position in enumerate(proposed_positions):
        if position is None or proposed_positions.count(position) > 1:
            grid[positions[index]] = "#"
            continue
        else:
            grid[position] = "#"

    if round == 9:
        min_row = min(np.argwhere(grid == "#"), key=itemgetter(0))[0]
        max_row = max(np.argwhere(grid == "#"), key=itemgetter(0))[0]
        min_col = min(np.argwhere(grid == "#"), key=itemgetter(1))[1]
        max_col = max(np.argwhere(grid == "#"), key=itemgetter(1))[1]
        print(f"Number of empty tiles after 10 rounds: {len(np.argwhere(grid[min_row:max_row+1,min_col:max_col+1] == '.'))}")

print(f"Number of rounds until no more movement: {round+1}")

Number of empty tiles after 10 rounds: 4249
Number of rounds until no more movement: 980
